In [14]:
from src.guess.guess import ConfigType, Guess, Config
from src.domain.datasets.UnixCommandDataset import UnixCommandDataset  # Register dataset
from src.domain.datasets.ProjectEulerDataset import ProjectEulerDataset
from src.domain.datasets.BringUpDataset import BringUpDataset

prediction = None

config = Config(ConfigType.QWEN_X862ARM64.get_path())
guess = Guess(config=config)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-23 01:06:30,076 - src.domain.models.QwenModel - INFO - Initializing QwenModel with ahmedheakl/asm2asm_1.5b_armv8_o0 on device mps


Loading Qwen model: ahmedheakl/asm2asm_1.5b_armv8_o0


2025-04-23 01:06:34,043 - QwenModel - INFO - Initialized QwenModel on device: mps:0
2025-04-23 01:06:34,044 - src.domain.models.QwenModel - INFO - Model initialization completed in 3.97 seconds


Source JSONL: data/processed/X86/BringUp_x86.jsonl
Target JSONL: data/processed/ARM64/BringUp_arm.jsonl
Loading file: data/processed/X86/BringUp_x86.jsonl
Loaded 48 entries from data/processed/X86/BringUp_x86.jsonl
Loading file: data/processed/ARM64/BringUp_arm.jsonl
Loaded 48 entries from data/processed/ARM64/BringUp_arm.jsonl


In [15]:
test_instance = guess.data_loader.iter().__next__()
print(test_instance)


Source entries: 48
Target entries: 48


KeyError: 'c_files'

In [3]:
import pickle
import os

if os.path.exists("test_pred.pkl"):
    with open("test_pred.pkl", "rb") as f:
        test_pred = pickle.load(f)
else:
    test_pred = guess.model.predict(test_instance, guess.inference_cfg)
    with open("test_pred.pkl", "wb") as f:
        pickle.dump(test_pred, f)

In [4]:
from src.sketch.sketch import Sketch

sketch = Sketch(config, guess.model)

In [5]:
import pandas as pd
from IPython.display import HTML, display

def html_escape(text):
    # Convert special characters to HTML-safe versions
    return (text.replace("&", "&amp;")
                .replace("<", "&lt;")
                .replace(">", "&gt;")
                .replace("\n", "<br>")
                .replace("\t", "&nbsp;&nbsp;&nbsp;&nbsp;"))  # 4 spaces for a tab

def display_blocks(blocks, pred):
    data = []

    for block in blocks:
        source_text = sketch.model.tokenizer.decode(
            pred.source[0][block.source_start:block.source_end]
        )
        pred_text = sketch.model.tokenizer.decode(
            pred.pred[0][block.pred_start:block.pred_end]
        )
        data.append({
            'SOURCE': html_escape(source_text),
            'PRED': html_escape(pred_text)
        })

    df = pd.DataFrame(data)

    styles = """
    <style>
    table {
        table-layout: auto;
        word-wrap: break-word;
    }
    td {
        white-space: normal !important;
        font-family: monospace;
        vertical-align: top;
    }
    </style>
    """
    display(HTML(styles + df.to_html(escape=False)))


In [9]:
# test_pred = predictions_test['UnixCommands/cat']
line_mappings = sketch.map_predicted_lines_to_source_lines(
    test_pred.source,
    test_pred.pred,
    test_pred.alignments
)

blocks = sketch.extract_pure_instruction_blocks(
    test_pred.source, test_pred.pred, line_mappings
)

print(blocks)

display_blocks(blocks, test_pred)

UnixCommands/cat
Failed to parse line [arm64]: 	bl(open@PLT

Failed to parse line [arm64]: 	bl(read@PLT

Failed to parse line [arm64]: 	bl_putchar@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	bl_close@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	bl_close@PLT

Error: too many values to unpack (expected 2)


,SOURCE,PRED
0,"addq $8, %rax","add x8, x8, #8"
1,"movq %rax, %rdi","mov x1, #0"
2,"movl $1, %edi","mov w0, #1"
3,"movq %rcx, %rsi movl %eax, %edi","mov x1, x3 mov x2, #99"
4,"movl %eax, %edi","mov x0, x8"


UnixCommands/cd


""


UnixCommands/cp
Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]: 	movk	w8, #49152, lsl #16

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9



...

,SOURCE,PRED
0,"subq $144, %rsp","sub sp, sp, #176"
1,"movq %rdx, %rsi","mov w8, #0"
2,"addq $8, %rax","add x8, x8, #8"
3,"addq $8, %rax","add x8, x8, #8"
4,"movq %rax, %rsi","mov x1, x8"


UnixCommands/touch
Failed to parse line [arm64]: 	blputs@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	blputs@PLT

Error: too many values to unpack (expected 2)


,SOURCE,PRED
0,"addq $8, %rax","add x8, x8, #8"
1,"addq $8, %rax","add x8, x8, #8"
2,"addq $8, %rax","add x8, x8, #8"
3,"movl $0, %esi","mov w1, #0"
4,"movl $0, %eax","mov w0, #0"


UnixCommands/xargs
Failed to parse line [arm64]: 	stur	x8, [x29, #-24]

Failed to parse line [arm64]: 	blfclose@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	blfclose@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	ldrsb	w8, [sp, #7]

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	ldrsb	w8, [sp, #7]

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	bl.Putchar@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	bl.Putchar@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	adds	w8, w8, #1

Error: too many values to unpack (expected 2)


,SOURCE,PRED
0,"subq $272, %rsp","sub sp, sp, #304"
1,"movl $100, %esi","mov w8, #0"
2,"movq %rax, %rsi","mov x1, x8"
3,"movq %rax, %rdi","mov x0, x8"
4,"movq %rax, %rdi","mov x0, x8"
5,"movl $100, %esi","mov x2, #0"
6,"movq %rax, %rdi","mov x0, x8"
7,"movl $100, %esi","mov w1, #100"
8,"movl %eax, %edi","mov x0, x8"
9,"movq %rax, %rdi","mov x0, x8"


In [7]:
results = sketch.sketch({"test_pred": test_pred})

for pred_result, sketch_result in results:
    print(pred_result.instance_id)
    print(f"Ratio of invalid_blocks: {len(sketch_result.invalid_blocks)/len(sketch_result.total_blocks)}")
    print(f"Ratio of non_equivalent_blocks: {len(sketch_result.non_equivalent_blocks)/len(sketch_result.total_blocks)}")

Failed to parse line [arm64]: 	bl(open@PLT

Failed to parse line [arm64]: 	bl(read@PLT

Failed to parse line [arm64]: 	bl_putchar@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	bl_close@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]: 	bl_close@PLT

Error: too many values to unpack (expected 2)
Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]: 	movk	w8, #49152, lsl #16

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [arm64]:                                         ; kill: def $x9 killed $w9

Failed to parse line [x86]: 	cmpb	$4, %al

Error: too many values to unpack (expected 2)
F

In [10]:
print(sketch.model.decode(test_pred.pred))

main:
.LFB6:
	.cfi_startproc
	stp	x29, x30, [sp, -160]!
	.cfi_def_cfa_offset 160
	.cfi_offset 29, -160
	.cfi_offset 30, -152
	mov	x29, sp
	str	w0, [sp, 28]
	str	x1, [sp, 16]
	adrp	x0, :got:__stack_chk_guard
	ldr	x0, [x0, #:got_lo12:__stack_chk_guard]
	ldr	x1, [x0]
	str	x1, [sp, 152]
	mov	x1, 0
	ldr	x0, [sp, 16]
	add	x0, x0, 8
	ldr	x0, [x0]
	mov	w1, 0
	bl	open
	str	w0, [sp, 40]
	b	.L2
.L3:
	add	x0, sp, 48
	ldr	w2, [sp, 44]
	mov	x1, x0
	mov	w0, 1
	bl	write
.L2:
	add	x0, sp, 48
	mov	x2, 99
	mov	x1, x0
	ldr	w0, [sp, 40]
	bl	read
	str	w0, [sp, 44]
	ldr	w0, [sp, 44]
	cmp	w0, 0
	bne	.L3
	mov	w0, 10
	bl	putchar
	ldr	w0, [sp, 40]
	bl	close
	mov	w0, 0
	mov	w1, w0
	adrp	x0, :got:__stack_chk_guard
	ldr	x0, [x0, #:got_lo12:__stack_chk_guard]
	ldr	x3, [sp, 152]
	ldr	x2, [x0]
	subs	x3, x3, x2
	mov	x2, 0
	beq	.L5
	bl	__stack_chk_fail
.L5:
	mov	w0, w1
	ldp	x29, x30, [sp], 160
	.cfi_restore 30
	.cfi_restore 29
	.cfi_def_cfa_offset 0
	ret
	.cfi_endproc
	.arch armv8-a
	.file	"program.c"
	.c"
	.text
	.alig